# 1. Set Hyper Parameters

In [38]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import os

In [39]:
learning_rate = 0.001
training_epochs = 15    #15epochs동안 학습함
batch_size = 100

#체크포인트를 저장할 디렉토리를 설정하는 부분
cur_dir = os.getcwd()
ckpt_dir_name = 'checkpoints'
model_dir_name = 'minst_cnn_seq'

checkpoint_dir = os.path.join(cur_dir, ckpt_dir_name, model_dir_name)
os.makedirs(checkpoint_dir, exist_ok = True)

checkpoint_prefix = os.path.join(checkpoint_dir, model_dir_name)

# 2. Make a Data Pipelining

In [40]:
mnist = keras. datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

#train과 test image는 255로 나누어서 0~1 값으로 스케일링 해줌
train_images = train_images.astype(np.float32) / 255.
test_images = test_images.astype(np.float32) / 255.

#train_image에 (60000, 28, 28)로 배치, height, width 3개가 있는데
#chanel이 빠져있다 따라서 axis = -1로 설정하여 마지막 차원에 차원을 하나 추가함
train_images = np.expand_dims(train_images, axis= -1)
test_images = np.expand_dims(test_images, axis= -1)

train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

#실제로 데이터를 짤라서 공급해주는 부분
train_dataset = tf.data.Dataset.from_tensor_slices((train_images,
            train_labels)).shuffle(buffer_size=100000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images,
            test_labels)).shuffle(buffer_size=100000).batch(batch_size)


# 3. Build a Neural Network Model - Sequential API

In [41]:
def create_model():
    model = keras.Sequential()
    #1번 째 레이어
    model.add(keras.layers.Conv2D(filters=32, kernel_size = 3, activation=tf.nn.relu,
                                 padding = 'SAME', input_shape=(28,28,1)))
    model.add(keras.layers.MaxPool2D(padding='SAME'))  #MaxPool
    #2번 째 레이어
    model.add(keras.layers.Conv2D(filters=64, kernel_size=3, activation=tf.nn.relu,
                                 padding='SAME'))
    model.add(keras.layers.MaxPool2D(padding='SAME'))   #MaxPool
    #3번 째 레이어
    model.add(keras.layers.Conv2D(filters=128, kernel_size=3, activation=tf.nn.relu,
                                 padding='SAME'))
    model.add(keras.layers.MaxPool2D(padding='SAME'))   #MaxPool
    
    #앞에까지 나왔던 featuremap을 벡터로 펴줌
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(256, activation=tf.nn.relu))
    #DenseLayout파라미터가 많기 때문에 Dropout을 적용함
    model.add(keras.layers.Dropout(0.4))
    model.add(keras.layers.Dense(10))
    return model

In [42]:
model = create_model()
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 7, 7, 128)         73856     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 4, 4, 128)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)             

# 4. Define a Loss Function
# 5. Calculate a Gradient

In [ ]:
def loss_fn(model, images, labels):
    logits = model(images, training=True) #training=True로 하면 Dropout한부분이 적용 된다.
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    labels=labels, logits=logits))
    return loss

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
        #back proposition으로 테입을 거꾸로 감듯이 거꾸로 감으면서 gradient를 계산해 준다
        return tape.gradient(loss, model.variables)

# 6. Select an Optimizer
# 7. Define a Metric for Model's Performance
# 8. Make a Checkpoint for Saving

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

def evaluate(model, images, labels):
    logits = model(images, training=False)  #training하는 단계가 아니기 때문에 False
    #정답과 결과의 argmax를 비교해서 같으면 맞고 다르면 틀림
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
    #위의 값의 평균을 구하면 accuracy가 나올 것임
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy

checkpoint = tf.train.Checkpoint(cnn=model)

# 9. Train and Validate a Neural Network Model

In [ ]:
for epoch in range(training_epochs):
    avg_loss=0.
    avg_train_acc=0.
    avg_test_acc=0.
    train_step=0
    test_step=0
    
    for images, labels in train_dataset:
        #계산한 gradient랑 모델의 variables들을 넣어주고
        #optimizer.apply를 사용하면 weight들이 업데이트가 된다
        grads = grad(model, images, labels)
        optimizer.apply_gradients(zip(grads, model.variables))
        #-----여기까지만 해도 학습은 된다------#
        
        loss = loss_fn(model, images, labels)
        acc = evaluate(model, images, labels)
        avg_loss = avg_loss + loss
        avg_train_acc = avg_train_acc + acc
        train_step += 1
    avg_loss = avg_loss / train_step
    avg_train_acc = avg_train_acc / train_step

In [ ]:
for images, labels in test_dataset:
    acc = evaluate(model, images, labels)
    avg_test_acc = avg_test_acc + acc
    test_step += 1
    avg_test_acc = avg_test_acc / test_step

    print('Epoch: ', '{}'.format(epoch+1), 'loss =', '{:.8f}'.format(avg_loss),
         'train accurancy = ', '{:.4f}'.format(avg_train_acc),
         'test accurancy = ', '{:.4f}'.format(avg_test_acc))
    #한 epoch이 끝날 때마다 체크포인트 설정
    checkpoint.save(file_prefix=checkpoint_prefix)